In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn import metrics
import time
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from math import sqrt
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization,LeakyReLU
from keras import optimizers,regularizers
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

from technicalSignals import momentum,SMA,inBBands


In [ ]:
tickers=['AAPL','AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX']
TREND_WINDOWs = [(-48,0),(-35,0),(-28,0),(-7,0),(-1,0),(1,2),(1,8),(1,29),(1,36),(1,50)]
TREND_WINDOWs = [(1,2),(1,8),(1,29),(1,36),(1,50)]
kind_of_dataset = 'AffectiveSpace'
NN_INPUT_DIM = 716

In [ ]:
class DatasetManager:
    def __init__(self):
        X_raw = None
        Y_raw = None
        Y = None
        X = None
    
    def load_dataset(self, ticker, kind, technicalFeatures=False):
        types = {'Summary': '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentFullAggregatedHourly/',
            'AffectiveSpace': '/home/simone/Desktop/NLFF/AffectiveSpace/Aggregated_AffectSummary_dataset/',
            'Title': '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentTitleAggregatedHourly/',
            'Senticnet':''}
        news =  pd.read_csv(types[kind]+ticker+'.csv')
        price = pd.read_csv('/home/simone/Desktop/NLFF/indexes/indexes'+ticker+'.csv')
        price = price.rename(index=str, columns={"date": "DATE"})
        news = news.rename(index=str, columns={"initTime": "DATE"})
        news = news.drop(['Unnamed: 0'], axis=1)
        news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in news['DATE']]
        # This datased is already GMT+0
        price['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in price['DATE']]
        if(technicalFeatures):
            price['mom_30'] = momentum(price, 30)
            price['mom_50'] = momentum(price, 50)
            price['mom_100'] = momentum(price, 100)
            price['mom_150'] = momentum(price, 150)
            price['SMA_30'] = SMA(price, 30)
            price['SMA_50'] = SMA(price, 50)
            price['SMA_100'] = SMA(price, 100)
            price['SMA_150'] = SMA(price, 150)
            price['in_BBands'] = inBBands(price)

        #ALLIGNMENT
        initDate = max(news['DATE'][0], datetime(2017, 5, 22, 0, 0, 0))
        finalDate = min(news['DATE'][len(news)-1],datetime(2018, 6, 20, 0, 0, 0))
        news.drop(news[news.DATE > finalDate].index, inplace=True)
        news.drop(news[news.DATE < initDate].index, inplace=True)
        news = news.reset_index(drop=True)
        price.drop(price[price.DATE > finalDate].index, inplace=True)
        price.drop(price[price.DATE < initDate].index, inplace=True)
        price = price.reset_index(drop=True)
        assert len(price) == len(news)
        # FEATURES
        sentiment = news.drop(['DATE'], axis=1)
        X = sentiment
        for window in [5,10,15,20,30,50]:
            temp = sentiment.rolling(window).mean()
            temp.columns = temp.columns +'_'+str(window)
            X = pd.concat([X, temp],axis=1)
        if(technicalFeatures):   
            technical_features = ['mom_30','mom_50','mom_100','mom_150','SMA_30','SMA_50','SMA_100','SMA_150','in_BBands']
            X = pd.concat([X, price[technical_features]],axis=1)

            
        #NORMALIZATION:
        min_max_scaler = preprocessing.MinMaxScaler()
        X = np.nan_to_num(np.asarray(X, dtype=float))
        X = np.asarray(min_max_scaler.fit_transform(X))
        self.X_raw = X
        self.Y_raw = price

    def get_dataset_for_trend(self, init, finish, perc_train = 0.7):
        y = list()
        x = list()
        dates = list()
        price = self.Y_raw
        for i in range(abs(init),len(price)-finish):
            cumulative_return =  (price.iloc[i+finish]['open']-price.iloc[i+init]['open'])/price.iloc[i+init]['open']
            s =np.sign(cumulative_return)
            y.append(0 if s==-1 else 1)
            dates.append(price.iloc[i]['DATE'])
            x.append(self.X_raw[i])

        y = np.array(y)
        x = np.array(x)
        self.X = x
        self.Y = y
        nt=math.ceil(len(x)*perc_train)
        x_tv = x[:nt]
        y_tv = y[:nt]
        x_test = x[nt:]
        y_test = y[nt:]
        dates_test = dates[nt:]
        return (x_tv,y_tv),(x_test,y_test),dates_test
    
    def get_dataset_for_trend_all_tickers(self, init, finish,kind, perc_train = 0.7, technicalFeatures=False):
        x_tv_all = []
        y_tv_all = []
        x_test_all = []
        y_test_all = []
        dates_test_prev = None
        for ticker in tickers:
            self.load_dataset(ticker, kind, technicalFeatures)
            (x_tv,y_tv),(x_test,y_test),dates_test = ds.get_dataset_for_trend(init, finish, perc_train = 0.7)
            if(dates_test_prev):
                assert dates_test == dates_test_prev #I'm not secure about this constraint but otherwise which dates I will output?
            x_tv_all += x_tv.tolist()
            y_tv_all += y_tv.tolist()
            x_test_all += x_test.tolist()
            y_test_all += y_test.tolist()
        x_tv_all = np.asarray(x_tv_all)
        y_tv_all = np.asarray(y_tv_all)
        x_test_all = np.asarray(x_test_all)
        y_test_all = np.asarray(y_test_all)
        return (x_tv_all,y_tv_all),(x_test_all,y_test_all), dates_test


        

In [ ]:
def buildNN(l2_regularizer,n_units,dropout,):
    model = Sequential()  
    model.add(Dense(n_units, input_dim=NN_INPUT_DIM, activity_regularizer=regularizers.l2(l2_regularizer))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dropout(dropout)) #According to dropout paper the probability oaver the input layer should be more than others
    model.add(Dense(math.floor(n_units/2), activity_regularizer=regularizers.l2(l2_regularizer))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(Dense(math.floor(n_units/4), activity_regularizer=regularizers.l2(l2_regularizer))) 
    model.add(BatchNormalization()) 
    model.add(LeakyReLU()) 
    model.add(Dense(1, activation='sigmoid')) 
    model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
    return model


def reset_weights(model):
    session=K.get_session()
    for layer in model.layers:
        if(hasattr(layer,'kernel_initializer')):
            layer.kernel.initializer.run(session=session)
            
def plot_hystory(history,l2,drop,n_units):
    f, axarr = plt.subplots(2, sharex=True)
    f.set_figheight(7)
    f.set_figwidth(10)
    axarr[0].plot(history.history['acc'],'g', label='accuracy_train')
    axarr[0].plot(history.history['val_acc'],'r', label='accuracy_val')
    axarr[1].semilogy(history.history['loss'],'g--',label='loss_train')
    axarr[1].semilogy(history.history['val_loss'],'r--',label='loss_val')
    #axarr[1].set_ylim([0,2])
    axarr[1].legend()   
    axarr[0].legend()   
    axarr[0].set_title('l2: '+str(l2)+' drop: '+str(drop)+' n_units: '+str(n_units))   
    plt.show() 
            
            
def cv(x_tv,y_tv):
    l2_space=[0.1,0.05,0.01]
    drop_space=[0.2,0.5,0.7]
    n_unit_space=[32,64,128,256]
    best_mcc = -float(np.inf)
    best_l2 = 0
    best_drop = 0
    best_n_units = 0
    for l2 in l2_space:
        for drop in drop_space:
            for n_units in n_unit_space:
                trainpoint=math.floor(len(x_tv)*0.70)
                dimval=math.floor(len(x_tv)*0.10)
                endval=trainpoint+dimval
                #Cross validation
                cvMCC = 0
                n_cv = 3
                nn_model = buildNN(l2_regularizer=l2,n_units=n_units,dropout=drop)
                for i in range(0,n_cv):
                    x_train=x_tv[0:trainpoint]
                    y_train=y_tv[0:trainpoint]
                    x_val=x_tv[trainpoint:endval]
                    y_val=y_tv[trainpoint:endval]
                    trainpoint=trainpoint+dimval
                    endval=endval+dimval
                    history = nn_model.fit(x_train, y_train, epochs = 400,batch_size =256, verbose=0, 
                                           validation_data=(x_val, y_val),shuffle=True)
                    y_pred = nn_model.predict(x_val, batch_size=128, verbose=0)
                    y_pred = [1 if y>0 else 0 for y in y_pred]
                    confmatrix = confusion_matrix(y_val, y_pred)
                    tn, fp, fn, tp = confmatrix.ravel()
                    denom = (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)
                    mcc = 0 if denom== 0 else (tp*tn -fp*fn)/sqrt(denom)
                    cvMCC += mcc/n_cv
                    reset_weights(nn_model)
                if(cvMCC > best_mcc):
                    best_mcc = cvMCC
                    best_l2 = l2
                    best_drop = drop
                    best_n_units = n_units
                plot_hystory(history,l2,drop,n_units)  
                
    return (best_l2,best_drop,best_n_units)

In [ ]:
for (init, finish) in TREND_WINDOWs:
    print('\n\n\n====================  trend: ',init,' ',finish, ' ==================== \n\n')
    ds = DatasetManager()
    (x_tv,y_tv),(x_test,y_test),dates_test = ds.get_dataset_for_trend_all_tickers(init,finish,kind_of_dataset,perc_train=0.7,technicalFeatures=True)
    (best_l2,best_drop,best_n_units) = cv(x_tv,y_tv)
    nn_model = buildNN(l2_regularizer=best_l2,n_units=best_n_units,dropout=best_drop)
    model_json = nn_model.to_json()
    with open("nn_model_"+str(init)+"_"+str(finish)+".json", "w") as json_file:
        json_file.write(model_json)
    history = nn_model.fit(x_tv, y_tv, epochs = 400,batch_size = 256, verbose=0, validation_data=(x_test, y_test),shuffle=True)
    nn_model.save_weights("nn_model_pretrained_weights_all_tickers_"+str(init)+"_"+str(finish)+".h5")
    y_pred = nn_model.predict(x_test, batch_size=256, verbose=0)
    print('==== Test ===')
    plot_hystory(history,best_l2,best_drop,best_n_units)  
    y_pred = [1 if y>0 else 0 for y in y_pred]
    acc = sum([1 if y_pred[i]*y_test[i]>0 else 0 for i in range(len(y_pred))])/len(y_pred)
    confmatrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = confmatrix.ravel()
    denom = (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)
    mcc = 0 if denom== 0 else (tp*tn -fp*fn)/sqrt(denom)
    print('test Acc: ',acc,' test MCC: ',mcc)